Subject's behavioural data during the training sessions and fMRI is preprocessed. The output of this pipeline is analysis dataframes

In [1]:
import os
import sys
from pathlib import Path
import glob
import getpass
import importlib

import numpy as np
import pandas as pd
import scipy.io as sio

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.cm import ScalarMappable
import matplotlib.cm as cm
import matplotlib.colors as mcolors

import utils
importlib.reload(utils)

# SET PATHS:
baseDir = os.path.join('/Users', getpass.getuser(), 'Desktop', 'Projects', 'bimanual_wrist')
fMRI_behavDir = os.path.join(baseDir,'data','fMRI','behavioural')
train_behavDir = os.path.join(baseDir,'data','training')
anaDir = os.path.join(baseDir,'analysis')
figDir = os.path.join(baseDir,'figures')

path = {'baseDir': baseDir,
        'fMRI_behavDir': fMRI_behavDir,
        'train_behavDir': train_behavDir,
        'anaDir': anaDir}



## Subject routine and prep

In [ ]:
sn = 7

# load the behavioural dta of subject:
train = pd.read_table(os.path.join(train_behavDir, f's{sn:02d}', f'BimanualWrist_MR_{sn}.dat'))
fMRI = pd.read_table(os.path.join(fMRI_behavDir, f's{sn:02d}', f'BimanualWrist_MR_{sn}.dat'))

utils.dataset.subject_routine(sn, path, smooth_win_sz=0, fs=200)


100%|██████████| 960/960 [00:56<00:00, 16.87it/s]


## all dataframe